<a href="https://colab.research.google.com/github/TaeGuSeo/Terms-and-Conditions-Analysis-System/blob/main/%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. **Import**:
   **`numpy`**는 수치 계산을 위한 라이브러리, **`itertools`**는 조합을 계산하기 위한 라이브러리, **`cosine_similarity`**는 코사인 유사도를 계산하기 위한 함수,
   **`SentenceTransformer`**는 문장 임베딩을 생성하기 위한 라이브러리, **`Kkma`**는 한국어 형태소 분석을 위한 라이브러리
2. **문서 및 명사 추출**:
   **`Kkma`**를 사용하여 문서를 형태소 단위로 분석하고 명사를 추출한 후, 불용어를 필터링하여 **`nouns`** 리스트에 저장.
3. **SentenceTransformer 모델 로드**:
   'distilbert-base-nli-mean-tokens' 모델을 로드. 이 모델은 문장을 입력으로 받아 각 문장의 임베딩을 생성하는데 사용됨.
4. **문서와 후보 키워드 임베딩 생성**:
   문서의 임베딩 **`doc_embedding`**을 생성합니다. 그리고 추출된 명사 리스트인 **`nouns`**에 대한 임베딩 **`candidate_embeddings`**을 생성.
5. **max_sum_sim 함수 정의**:
주어진 문서 임베딩과 후보 키워드 임베딩 간의 코사인 유사도를 계산하고, 그 중 가장 관련성이 높은 주요 키워드를 선택하는것.
    - **`distances`**: 문서 임베딩과 후보 키워드 임베딩 간의 코사인 유사도를 계산.
    - **`distances_candidates`**: 후보 키워드 간의 코사인 유사도를 계산.
    - **`words_idx`**: 후보 키워드 중에서 유사도가 가장 높은 상위 **`nr_candidates`** 개의 인덱스를 선택.
    - **`distances_candidates`** 행렬은 후보 키워드 간의 유사도.
    이후에는 후보 키워드 간의 조합을 모두 생성하고, 조합의 유사도 합을 계산하여 최적의 조합을 선택. 이렇게 선택된 후보 키워드 조합이 최종적인 주요 키워드로 반환.
    
6. **주요 키워드 추출**:
   **`max_sum_sim`** 함수를 사용하여 주요 키워드를 추출. **`top_keywords`** 변수에 추출된 주요 키워드가 저장.
7. **결과 출력**:
   추출된 주요 키워드를 출력.


**유사도 검출에 TF-IDF 사용:**

TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화는 텍스트 데이터를 수치화하는 방법 중 하나로, 텍스트 문서를 단어의 빈도와 역문서 빈도의 곱으로 표현함. 이를 통해 각 문서의 단어들이 전체 문서 집합에서 얼마나 중요한 단어인지를 나타냄.

각 문서의 단어들이 전체 문서들에서 얼마나 흔하게 나타나는지(TF)와 전체 문서 집합에서 얼마나 흔하게 나타나지 않는 단어인지(IDF)를 고려하여 단어들을 수치로 표현함.

특정 문서에 단어 "apple"이 5번 나타났고, 전체 문서 집합에 "apple"이 포함된 문서의 개수가 100개라면 해당 문서의 "apple"에 해당하는 TF-IDF 값은 5 * log(100/100) = 5 * 0 = 0이 된다.

흐름 :

1. 이미지 텍스트와 법률 문장들을 하나의 리스트로 합치고, 이를 TF-IDF 벡터화하여 텍스트 데이터를 수치화한다
2. 이미지 텍스트와 법률 문장들 간의 코사인 유사도를 계산하여 유사도 행렬을 얻는다
3. 이 유사도 행렬을 기반으로 유사도가 높은 순서대로 정렬하여 일정 개수의 인덱스를 추출한다
4. 이렇게 추출된 인덱스는 사용자에게 주어진다

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install necessary packages and language data
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-kor
!pip install konlpy

from google.colab import files
import pytesseract
from PIL import Image
from konlpy.tag import Kkma
import re

def extract_keywords(text):
    # 형태소 분석기를 이용하여 명사 추출
    kkma = Kkma()
    nouns = kkma.nouns(text)
    return nouns

def model7(image_file):
    # 이미지 열기
    image = Image.open(image_file)

    # OCR 적용
    extracted_text = pytesseract.image_to_string(image, lang='kor')

    # 문장 부호 제거
    extracted_text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', extracted_text)

    # 주요한 키워드 추출 (명사 단위)
    nouns = extract_keywords(extracted_text)

    # 숫자가 포함된 단어와 단어 길이가 1인 것, 중복 단어 제거
    meaningful_nouns = list(set([noun for noun in nouns if len(noun) > 1 and not bool(re.search(r'\d', noun))]))

    # 불용어 리스트
    stopwords = ['보험', '약관', '부분','기타']
    meaningful_nouns = [noun for noun in meaningful_nouns if noun not in stopwords]

    # 상위 6개 단어 추출
    top_nouns = sorted(meaningful_nouns, key=lambda x: nouns.count(x), reverse=True)[:6]

    return top_nouns

# Google Colab에서 파일 업로드
uploaded_image = files.upload()

# 이미지 파일명을 아래의 변수에 지정합니다.
image_file = list(uploaded_image.keys())[0]

# '모델7' 호출하여 이미지에서 주요 키워드 추출
top_nouns = model7(image_file)

# 추출된 상위 6개 명사 출력
print("주요 키워드:")
print(top_nouns)0